In [1]:
import gurobipy as gp

# Primeira execução

## Read Files 1

In [ ]:
file = open('17.txt')
lines = file.readlines()
file.close()

cost_matrix = []

for i in range(len(lines)):
    aux = lines[i][:-1].split('\t')
    aux = [int(i) for i in aux if i!= '']
    cost_matrix.append(aux)

tam = len(cost_matrix)   
n = list(range(tam))
m = list(range(1, tam))

for i in range(tam):
    cost_matrix[i][i] = 999999999999

## Modelo Normal para o Problema do Caixeiro Viajante (PCV/TSP)

In [ ]:
import gurobipy as gp

model = gp.Model()

# Insere as variáveis de decisão
x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

# Define a função objetivo
model.setObjective(
    gp.quicksum(x[i, j] * cost_matrix[i][j] for i in n for j in n),
    sense=gp.GRB.MINIMIZE)

# Restrição 1
c1 = model.addConstrs(
    gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

# Restrição 2
c2 = model.addConstrs(
    gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

# Restrição 3
c3 = model.addConstrs(u[i] - u[j] + tam * x[i, j] <= tam-1 
                          for i in m 
                          for j in n 
                          if (i != j))

## Execução do modelo de forma normal

In [ ]:
# Executa o modelo
model.optimize()

## Exibe a solução

In [ ]:
# Valor objetivo
print("Valor Total:", model.objVal)

# Mostra os itens nas mochilas
resultado = 0
for i in n:
    for j in n:
        if x[i, j].x != 0:
            print(i, j)
            print('-->', x[i, j].x)
            resultado = resultado + cost_matrix[i][j]

Valor Total: 39.0
0 11
--> 1.0
1 9
--> 1.0
2 13
--> 1.0
3 16
--> 1.0
4 3
--> 1.0
5 15
--> 1.0
6 5
--> 1.0
7 1
--> 1.0
8 7
--> 1.0
9 10
--> 1.0
10 12
--> 1.0
11 6
--> 1.0
12 2
--> 1.0
13 0
--> 1.0
14 4
--> 1.0
15 14
--> 1.0
16 8
--> 1.0


# Testes PI

## Read Files

In [1]:
import pandas as pd

urls = [
    'TSP_3160_9998.xlsx',
    'TSP_3160_99971.xlsx',
    'TSP_3160_999876.xlsx',
    'TSP_3160_9999763.xlsx',
    'TSP_4005_9999.xlsx',
    'TSP_4005_99982.xlsx',
    'TSP_4005_999486.xlsx',
    'TSP_4005_9999932.xlsx',
    'TSP_4950_9998.xlsx',
    'TSP_4950_99983.xlsx',
    'TSP_4950_999935.xlsx',
    'TSP_4950_9998865.xlsx',
    'TSP_5995_9999.xlsx',
    'TSP_5995_99971.xlsx',
    'TSP_5995_999897.xlsx',
    'TSP_5995_9999030.xlsx',
]

dist = []
tamanhos = []
for url in urls:
  data = pd.read_excel(url)

  dist.append({(c['Origens'], c['Destinos']): c['Custos'] for c in data.to_dict('records')})

  tamanhos.append(data.Destinos.max()+1)

In [2]:
costs_matrix = []
for i in tamanhos:
  aux1 = []
  for j in range(i):
    aux2 = []
    for j in range(i):
      aux2.append(99999999999999999)
    aux1.append(aux2)
  costs_matrix.append(aux1)
cont = 0

for i in dist:
  for j, k in i:
    costs_matrix[cont][j][k] = i[j, k]
    costs_matrix[cont][k][j] = i[j, k]
  cont = cont + 1

## Pre-Solver

In [6]:
import gurobipy as gp

cont = 0

solucoes_pre = []
tempo_sol_pre = []

for k in tamanhos:
  n = list(range(k))
  m = list(range(1, k))

  model = gp.Model()
  #Limitar a busca em 10 minutos
  model.setParam(gp.GRB.Param.TimeLimit, 600)

  # Insere as variáveis de decisão
  x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
  u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

  # Define a função objetivo
  model.setObjective(
      gp.quicksum(x[i, j] * costs_matrix[cont][i][j] for i in n for j in n),
      sense=gp.GRB.MINIMIZE)

  # Restrição 1
  c1 = model.addConstrs(
      gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

  # Restrição 2
  c2 = model.addConstrs(
      gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

  # Restrição 3
  c3 = model.addConstrs(u[i] - u[j] + k * x[i, j] <= k-1 
                            for i in m 
                            for j in n 
                            if (i != j))

  model.optimize()

  cont = cont+1

  solucoes_pre.append(model.objVal)
  tempo_sol_pre.append(model.Runtime)
    
  print()
  print()
  print()
  print('---------------------------------------------------------------')
  print("Execução: ", cont)
  print('---------------------------------------------------------------')
  print()
  print()
  print()

Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0xb3a2210d
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 79 rows and 1 columns
Presolve time: 0.05s
Presolved: 6322 rows, 6479 columns, 31286 nonzeros
Variable types: 0 continuous, 6479 integer (6400 binary)

Root relaxation: objective 4.081044e+05, 371 iterations, 0.03 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0x6a6c2fa8
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+05, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 79 rows and 1 columns
Presolve time: 0.04s
Presolved: 6322 rows, 6479 columns, 31286 nonzeros
Variable types: 0 continuous, 6479 integer (6400 binary)

Root relaxation: objective 4.066881e+07, 418 iterations, 0.03 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.0669e+07    0   90       

H 1086   536                    4.108258e+08 4.1069e+08  0.03%  33.6   20s
H 1092   512                    4.107537e+08 4.1069e+08  0.02%  33.4   21s
  1098   517 4.1075e+08  102   80 4.1075e+08 4.1069e+08  0.02%  37.8   25s
  1125   535 4.1074e+08   27  230 4.1075e+08 4.1070e+08  0.01%  36.9   30s

Cutting planes:
  Learned: 27
  Gomory: 45
  MIR: 11
  Flow cover: 131
  Zero half: 47
  RLT: 1

Explored 1135 nodes (51035 simplex iterations) in 30.31 seconds (21.16 work units)
Thread count was 8 (of 8 available processors)

Solution count 6: 4.10754e+08 4.10826e+08 4.1083e+08 ... 4.11195e+08

Optimal solution found (tolerance 1.00e-04)
Best objective 4.107537220000e+08, best bound 4.107146140000e+08, gap 0.0095%



---------------------------------------------------------------
Execução:  4
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logica

     0     0 4579604.18    0  218          - 4579604.18      -     -    0s
     0     0 4580422.16    0  222          - 4580422.16      -     -    0s
     0     0 4580429.22    0  234          - 4580429.22      -     -    0s
     0     0 4580429.22    0  225          - 4580429.22      -     -    0s
     0     0 4580439.00    0  251          - 4580439.00      -     -    1s
H    0     0                    4593004.0000 4580439.00  0.27%     -    1s
     0     0 4580439.00    0  228 4593004.00 4580439.00  0.27%     -    2s
     0     0 4580439.00    0   84 4593004.00 4580439.00  0.27%     -    2s
     0     0 4580439.00    0  113 4593004.00 4580439.00  0.27%     -    2s
     0     0 4580439.00    0  249 4593004.00 4580439.00  0.27%     -    2s
     0     0 4580439.00    0  231 4593004.00 4580439.00  0.27%     -    2s
     0     0 4580439.00    0  234 4593004.00 4580439.00  0.27%     -    2s
     0     0 4580439.00    0  241 4593004.00 4580439.00  0.27%     -    3s
     0     0 4580439.00  

     0     0 4.5980e+07    0  253 4.6006e+07 4.5980e+07  0.06%     -    9s
     0     0 4.5980e+07    0  227 4.6006e+07 4.5980e+07  0.06%     -    9s
     0     0 4.5980e+07    0  204 4.6006e+07 4.5980e+07  0.06%     -    9s
     0     0 4.5980e+07    0  236 4.6006e+07 4.5980e+07  0.06%     -    9s
     0     0 4.5980e+07    0  233 4.6006e+07 4.5980e+07  0.06%     -    9s
     0     0 4.5980e+07    0  128 4.6006e+07 4.5980e+07  0.06%     -    9s
     0     0 4.5980e+07    0  140 4.6006e+07 4.5980e+07  0.06%     -    9s
     0     0 4.5980e+07    0   36 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  211 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  119 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  209 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0   55 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  275 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07  

Presolve time: 0.07s
Presolved: 9902 rows, 10099 columns, 49106 nonzeros
Variable types: 0 continuous, 10099 integer (10000 binary)

Root relaxation: objective 5.069919e+05, 443 iterations, 0.03 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 506991.890    0  144          - 506991.890      -     -    0s
     0     0 508283.460    0  254          - 508283.460      -     -    1s
H    0     0                    508355.00000 508283.460  0.01%     -    1s
     0     0     cutoff    0      508355.000 508355.000  0.00%     -    1s

Cutting planes:
  Learned: 38
  MIR: 62
  Zero half: 1

Explored 1 nodes (1140 simplex iterations) in 1.50 seconds (0.70 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 508355 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.083550000000e+05, best bound 5.083550000000e+05, gap 0.00

     0     0 5.1136e+07    0  262 5.1296e+07 5.1136e+07  0.31%     -   21s
     0     0 5.1136e+07    0  262 5.1296e+07 5.1136e+07  0.31%     -   21s
     0     0 5.1136e+07    0   91 5.1296e+07 5.1136e+07  0.31%     -   21s
     0     0 5.1136e+07    0  278 5.1296e+07 5.1136e+07  0.31%     -   22s
     0     0 5.1136e+07    0  246 5.1296e+07 5.1136e+07  0.31%     -   22s
     0     0 5.1136e+07    0  251 5.1296e+07 5.1136e+07  0.31%     -   22s
     0     0 5.1136e+07    0  232 5.1296e+07 5.1136e+07  0.31%     -   22s
     0     0 5.1136e+07    0  230 5.1296e+07 5.1136e+07  0.31%     -   23s
     0     2 5.1136e+07    0  103 5.1296e+07 5.1136e+07  0.31%     -   24s
     7    12 5.1136e+07    3  215 5.1296e+07 5.1136e+07  0.31%   205   25s
H   29    33                    5.117018e+07 5.1136e+07  0.07%   139   29s
*   62    62              14    5.113695e+07 5.1136e+07  0.00%   109   29s

Cutting planes:
  Learned: 41
  Gomory: 10
  MIR: 173

Explored 63 nodes (12635 simplex iterations)

H    0     0                    563875.00000 560017.000  0.68%     -   19s
     0     0 560017.000    0  192 563875.000 560017.000  0.68%     -   19s
     0     0 560017.000    0   59 563875.000 560017.000  0.68%     -   19s
     0     0 560017.000    0  168 563875.000 560017.000  0.68%     -   20s
     0     0 560017.000    0  296 563875.000 560017.000  0.68%     -   20s
     0     0 560017.000    0  280 563875.000 560017.000  0.68%     -   22s
     0     0 560017.000    0  283 563875.000 560017.000  0.68%     -   22s
     0     0 560017.000    0  325 563875.000 560017.000  0.68%     -   22s
     0     0 560017.000    0  326 563875.000 560017.000  0.68%     -   22s
     0     0 560017.000    0  328 563875.000 560017.000  0.68%     -   23s
     0     0 560017.000    0  326 563875.000 560017.000  0.68%     -   23s
     0     0 560017.000    0  171 563875.000 560017.000  0.68%     -   24s
     0     0 560017.000    0  240 563875.000 560017.000  0.68%     -   25s
     0     0 560017.000  

  1725  1641 5.5962e+07   35  291 5.5993e+07 5.5957e+07  0.06%  26.5   51s
  1904  1780 5.5963e+07   52  259 5.5993e+07 5.5957e+07  0.06%  31.1   56s
H 1982  1709                    5.599172e+07 5.5957e+07  0.06%  32.4   68s
  2016  1805 5.5967e+07   62  211 5.5992e+07 5.5957e+07  0.06%  32.9   71s
* 2093  1691              77    5.598643e+07 5.5957e+07  0.05%  35.3   71s
* 2185  1533              78    5.597602e+07 5.5957e+07  0.03%  35.5   73s
  2350  1464 5.5964e+07   94  265 5.5976e+07 5.5957e+07  0.03%  36.4   75s
  2596  1561 5.5962e+07   17  100 5.5976e+07 5.5957e+07  0.03%  37.3   83s
  2914  1845     cutoff   44      5.5976e+07 5.5957e+07  0.03%  37.0   85s
  3658  2244     cutoff   44      5.5976e+07 5.5957e+07  0.03%  35.9   92s
H 4783  1137                    5.596637e+07 5.5958e+07  0.02%  35.0   94s
  4784  1136 5.5964e+07   34  274 5.5966e+07 5.5958e+07  0.02%  35.0   95s
  4787  1138 5.5965e+07   48  318 5.5966e+07 5.5958e+07  0.02%  34.9  102s
  4794  1143 5.5964e+07  

## Desabilita pre-solver

In [7]:
import gurobipy as gp

cont = 0

solucoes_no_pre = []
tempo_sol_no_pre = []

for k in tamanhos:
  n = list(range(k))
  m = list(range(1, k))

  model = gp.Model()
  #Limitar a busca em 10 minutos
  model.setParam(gp.GRB.Param.TimeLimit, 600)
    
  # Insere as variáveis de decisão
  x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
  u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

  # Define a função objetivo
  model.setObjective(
      gp.quicksum(x[i, j] * costs_matrix[cont][i][j] for i in n for j in n),
      sense=gp.GRB.MINIMIZE)

  # Restrição 1
  c1 = model.addConstrs(
      gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

  # Restrição 2
  c2 = model.addConstrs(
      gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

  # Restrição 3
  c3 = model.addConstrs(u[i] - u[j] + k * x[i, j] <= k-1 
                            for i in m 
                            for j in n 
                            if (i != j))


  # Desativa o Pre-solver
  model.setParam(gp.GRB.Param.Presolve, 0)
  # Executa o modelo
  model.optimize()

  cont = cont+1

  solucoes_no_pre.append(model.objVal)
  tempo_sol_no_pre.append(model.Runtime)

  print()
  print()
  print()
  print('---------------------------------------------------------------')
  print("Execução: ", cont)
  print('---------------------------------------------------------------')
  print()
  print()
  print()

Set parameter TimeLimit to value 600
Set parameter Presolve to value 0
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0xb3a2210d
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Variable types: 0 continuous, 6480 integer (6400 binary)

Root relaxation: objective 4.081044e+05, 373 iterations, 0.03 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 408104.375    0  106          - 408104.375      -     -    0s
     

     0     0 4.0812e+07    0  199          - 4.0812e+07      -     -    1s
     0     0 4.0812e+07    0  158          - 4.0812e+07      -     -    1s
     0     0 4.0812e+07    0  228          - 4.0812e+07      -     -    1s
     0     0 4.0812e+07    0  156          - 4.0812e+07      -     -    1s
     0     0 4.0812e+07    0  195          - 4.0812e+07      -     -    1s
     0     0 4.0812e+07    0  197          - 4.0812e+07      -     -    2s
     0     0 4.0812e+07    0  206          - 4.0812e+07      -     -    2s
     0     0 4.0812e+07    0  205          - 4.0812e+07      -     -    2s
     0     0 4.0812e+07    0  198          - 4.0812e+07      -     -    2s
H    0     0                    4.095442e+07 4.0812e+07  0.35%     -    2s
     0     0 4.0812e+07    0  193 4.0954e+07 4.0812e+07  0.35%     -    2s
     0     0 4.0812e+07    0  170 4.0954e+07 4.0812e+07  0.35%     -    2s
     0     2 4.0812e+07    0  138 4.0954e+07 4.0812e+07  0.35%     -    3s
H   30    35             

  1084   557 459466.104   45  256 459649.000 459428.453  0.05%  27.5   50s
  1087   559 459431.949   49  215 459649.000 459431.949  0.05%  27.4   55s
  1095   565 459649.000   32  203 459649.000 459434.072  0.05%  27.2   60s
  1100   568 459649.000   82  251 459649.000 459436.936  0.05%  27.1   65s
  1105   571 459438.653   21  187 459649.000 459438.653  0.05%  27.0   74s
  1106   575 459438.653   12  250 459649.000 459438.653  0.05%  36.7   76s
H 1135   566                    459543.00000 459438.653  0.02%  37.8   77s
  1255   614 459438.653   25  175 459543.000 459438.653  0.02%  40.0   80s
  1631   794 459519.348   22  214 459543.000 459438.653  0.02%  39.7   86s
  2248   745 459515.054   28  141 459543.000 459438.653  0.02%  42.6   90s
* 2535   597              28    459506.00000 459438.653  0.01%  42.8   93s
  2633   530 459476.735   28  217 459506.000 459438.653  0.01%  43.1   95s
* 2866   469              31    459496.00000 459438.653  0.01%  44.3   95s
  3428   398 459455.515  




---------------------------------------------------------------
Execução:  7
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter Presolve to value 0
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8101 rows, 8190 columns and 39963 nonzeros
Model fingerprint: 0xb639b0ef
Variable types: 0 continuous, 8190 integer (8100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+06, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Variable types: 0 continuous, 8190 integer (8100 binary)

Root relaxation: objective 4.595493e+08, 455 iterations, 0.03 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexp

         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Variable types: 0 continuous, 10100 integer (10000 binary)

Root relaxation: objective 5.090122e+07, 542 iterations, 0.03 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5.0901e+07    0  141          - 5.0901e+07      -     -    0s
     0     0 5.1116e+07    0  261          - 5.1116e+07      -     -    0s
     0     0 5.1120e+07    0  256          - 5.1120e+07      -     -    1s
     0     0 5.1120e+07    0  270          - 5.1120e+07      -     -    1s
     0     0 5.1136e+07    0  291          - 5.1136e+07      -     -    2s
     0     0 5.1136e+07    0  291          - 5.1136e+07      -     -    5s
     0     0 5.1136e+07    0  285          - 5.1136e+07      -     -    5s
H    0     0                    5.281040e+07 5.1136e+07  3.17%     


Optimal solution found (tolerance 1.00e-04)
Best objective 5.600210000000e+05, best bound 5.600210000000e+05, gap 0.0000%



---------------------------------------------------------------
Execução:  13
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter Presolve to value 0
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12101 rows, 12210 columns and 59843 nonzeros
Model fingerprint: 0x3988ee64
Variable types: 0 continuous, 12210 integer (12100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+04, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Variable types: 0 continuous, 12210 integer (12100 binary)

Root relaxation: objective 5.581193e+06, 60

  3804  2029 5.5970e+07   41  134 5.5979e+07 5.5958e+07  0.04%  29.8  136s
  4474  2202     cutoff   62      5.5979e+07 5.5958e+07  0.04%  30.1  141s
  5016  2405 5.5977e+07   50   42 5.5979e+07 5.5958e+07  0.04%  31.9  147s
  5410  2371 5.5960e+07   76  114 5.5979e+07 5.5958e+07  0.04%  32.7  162s
  5574  2357 5.5960e+07   91  120 5.5979e+07 5.5958e+07  0.04%  32.5  167s
  5671  2641     cutoff  102      5.5979e+07 5.5958e+07  0.04%  32.8  170s
  6650  3288 5.5959e+07   31  216 5.5979e+07 5.5958e+07  0.04%  32.2  178s
  7214  3645 5.5965e+07   32  273 5.5979e+07 5.5959e+07  0.04%  32.5  182s
* 7602  3171              43    5.597333e+07 5.5959e+07  0.03%  32.8  182s
  7753  3426 5.5971e+07   27  210 5.5973e+07 5.5959e+07  0.03%  32.6  185s
  8323  3797     cutoff   60      5.5973e+07 5.5959e+07  0.03%  32.0  190s
  9505  4566 5.5961e+07   21  134 5.5973e+07 5.5959e+07  0.02%  31.8  198s
 10224  5065 5.5967e+07   41  229 5.5973e+07 5.5960e+07  0.02%  31.8  204s
*10261  3749             

## Model for Lazy Constraints And CallBack (c4 in model)

In [8]:
import gurobipy as gp

cont = 0

solucoes_NCB = []
tempo_NCB = []

for k in tamanhos:
  n = list(range(k))
  m = list(range(1, k))

  model = gp.Model()
  #Limitar a busca em 10 minutos
  model.setParam(gp.GRB.Param.TimeLimit, 600)

  # Insere as variáveis de decisão
  x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
  u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

  # Define a função objetivo
  model.setObjective(
      gp.quicksum(x[i, j] * costs_matrix[cont][i][j] for i in n for j in n),
      sense=gp.GRB.MINIMIZE)

  # Restrição 1
  c1 = model.addConstrs(
      gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

  # Restrição 2
  c2 = model.addConstrs(
      gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

  # Restrição 3
  c3 = model.addConstrs(u[i] - u[j] + k * x[i, j] <= k-1 
                            for i in m 
                            for j in n 
                            if (i != j))

  c4 = model.addConstr(x[0, 1] == 1)
    
    
  # Executa o modelo
  model.optimize()

  cont = cont+1

  print()
  print()
  print()
  print('---------------------------------------------------------------')
  print("Execução: ", cont)
  print('---------------------------------------------------------------')
  print()
  print()
  print()  

  solucoes_NCB.append(model.objVal)
  tempo_NCB.append(model.Runtime)




Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6402 rows, 6480 columns and 31524 nonzeros
Model fingerprint: 0x827ccb5c
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 82 rows and 160 columns
Presolve time: 0.05s
Presolved: 6320 rows, 6320 columns, 30890 nonzeros
Variable types: 0 continuous, 6320 integer (6241 binary)

Root relaxation: objective 4.105508e+05, 374 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

     0     0 4.1183e+07    0   55          - 4.1183e+07      -     -    1s
H    0     0                    7.800000e+18 4.1183e+07   100%     -    1s
     0     0 4.1183e+07    0   55 7.8000e+18 4.1183e+07   100%     -    2s
H    0     0                    4.132333e+07 4.1183e+07  0.34%     -    3s
     0     0 4.1183e+07    0  104 4.1323e+07 4.1183e+07  0.34%     -    3s
     0     0 4.1183e+07    0  183 4.1323e+07 4.1183e+07  0.34%     -    4s
     0     0 4.1183e+07    0   56 4.1323e+07 4.1183e+07  0.34%     -    4s
     0     0 4.1183e+07    0   58 4.1323e+07 4.1183e+07  0.34%     -    4s
     0     0 4.1183e+07    0   68 4.1323e+07 4.1183e+07  0.34%     -    4s
     0     0 4.1183e+07    0   68 4.1323e+07 4.1183e+07  0.34%     -    4s
     0     0 4.1183e+07    0   70 4.1323e+07 4.1183e+07  0.34%     -    4s
     0     0 4.1183e+07    0   70 4.1323e+07 4.1183e+07  0.34%     -    4s
     0     0 4.1183e+07    0   70 4.1323e+07 4.1183e+07  0.34%     -    4s
     0     0 4.1183e+07  

     0     0 464270.000    0   22          - 464270.000      -     -    2s
     0     0 464270.000    0   22          - 464270.000      -     -    2s
     0     0 464270.000    0   22          - 464270.000      -     -    3s
     0     0 464270.000    0   22          - 464270.000      -     -    3s
H    0     0                    8.800000e+18 464270.000   100%     -    3s
     0     0 464270.000    0   22 8.8000e+18 464270.000   100%     -    3s
H    0     0                    9.000000e+17 464270.000   100%     -    5s
     0     2 464270.000    0   22 9.0000e+17 464270.000   100%     -    5s
    15    20 464357.478    4   51 9.0000e+17 464271.198   100%  66.1   10s
    27    32 464281.140    7  143 9.0000e+17 464271.198   100%  65.3   16s
H   28    32                    465879.00000 464271.198  0.35%  62.9   16s
H   29    32                    465382.00000 464271.198  0.24%  61.8   16s
H   30    32                    464547.00000 464271.198  0.06%  60.7   16s
*  322   179             


Solution count 8: 4.64336e+07 4.64449e+07 4.64449e+07 ... 8.8e+18

Optimal solution found (tolerance 1.00e-04)
Best objective 4.643361500000e+07, best bound 4.643147646154e+07, gap 0.0046%



---------------------------------------------------------------
Execução:  7
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8102 rows, 8190 columns and 39964 nonzeros
Model fingerprint: 0xda4bde35
Variable types: 0 continuous, 8190 integer (8100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+06, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 92 rows and 180 columns
Presolve time: 0.06s
Presolved: 80

Presolved: 9900 rows, 9900 columns, 48610 nonzeros
Variable types: 0 continuous, 9900 integer (9801 binary)

Root relaxation: objective 5.135970e+06, 516 iterations, 0.03 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5135969.76    0  120          - 5135969.76      -     -    0s
     0     0 5151397.14    0  220          - 5151397.14      -     -    1s
H    0     0                    5152573.0000 5152313.30  0.01%     -    1s
     0     0 5152573.00    0  260 5152573.00 5152573.00  0.00%     -    1s

Cutting planes:
  Learned: 32
  MIR: 48

Explored 1 nodes (1321 simplex iterations) in 1.59 seconds (0.82 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 5.15257e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.152573000000e+06, best bound 5.152573000000e+06, gap 0.0000%



---------------------------


Root relaxation: objective 5.626465e+05, 580 iterations, 0.06 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 562646.491    0  137          - 562646.491      -     -    0s
     0     0 564549.600    0  214          - 564549.600      -     -    1s
     0     0 564561.418    0  230          - 564561.418      -     -    2s
     0     0 564561.436    0  228          - 564561.436      -     -    2s
     0     0 564585.000    0  232          - 564585.000      -     -    2s
     0     0 564585.000    0  244          - 564585.000      -     -    3s
     0     0 564585.000    0  253          - 564585.000      -     -    3s
     0     0 564585.000    0  253          - 564585.000      -     -    3s
     0     0 564585.667    0  116          - 564585.667      -     -    4s
     0     0 564585.667    0  161          - 564585.667      -     -    4s
     0   

Presolve time: 0.11s
Presolved: 11990 rows, 11990 columns, 58970 nonzeros
Variable types: 0 continuous, 11990 integer (11881 binary)

Root relaxation: objective 5.595271e+07, 604 iterations, 0.06 seconds (0.06 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5.5953e+07    0  137          - 5.5953e+07      -     -    0s
     0     0 5.6099e+07    0  255          - 5.6099e+07      -     -    2s
     0     0 5.6101e+07    0  255          - 5.6101e+07      -     -    2s
     0     0 5.6101e+07    0  253          - 5.6101e+07      -     -    2s
     0     0 5.6102e+07    0  250          - 5.6102e+07      -     -    2s
     0     0 5.6102e+07    0  248          - 5.6102e+07      -     -    3s
*    0     0               0    5.610262e+07 5.6103e+07  0.00%     -    3s

Cutting planes:
  Learned: 33
  Gomory: 12
  MIR: 114
  Mod-K: 8
  RLT: 13

Explored 1 nodes (1970 s

## Using CB and LC

In [3]:
import gurobipy as gp

cont = 0

solucoes_CB = []
tempo_CB = []

for k in tamanhos:
  n = list(range(k))
  m = list(range(1, k))

  model = gp.Model()
  #Limitar a busca em 10 minutos
  model.setParam(gp.GRB.Param.TimeLimit, 600)

  # Insere as variáveis de decisão
  x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
  u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

  # Define a função objetivo
  model.setObjective(
      gp.quicksum(x[i, j] * costs_matrix[cont][i][j] for i in n for j in n),
      sense=gp.GRB.MINIMIZE)

  # Restrição 1
  c1 = model.addConstrs(
      gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

  # Restrição 2
  c2 = model.addConstrs(
      gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

  # Restrição 3
  c3 = model.addConstrs(u[i] - u[j] + k * x[i, j] <= k-1 
                            for i in m 
                            for j in n 
                            if (i != j))

  def newRoute(model, where):
    if where == gp.GRB.Callback.MIPSOL:
        print('----------------------------------------------------------------')
        model.cbLazy(x[0, 1] == 1)
    
  cont = cont+1

  print()
  print()
  print()
  print('---------------------------------------------------------------')
  print("Execução: ", cont+1)
  print('---------------------------------------------------------------')
  print()
  print()
  print()
  
  # Executa o modelo
  model.Params.lazyConstraints = 1
  model.optimize(newRoute)

  solucoes_CB.append(model.objVal)
  tempo_CB.append(model.Runtime)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-01
Set parameter TimeLimit to value 600



---------------------------------------------------------------
Execução:  2
---------------------------------------------------------------



Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0xb3a2210d
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.21s
Presolved: 6401 rows, 6480 columns, 31523 nonzeros
Variable types: 0 continuous, 6480 integer (6400 binary)

Roo

     0     0 4.0812e+07    0  202          - 4.0812e+07      -     -    2s
     0     0 4.0812e+07    0  201          - 4.0812e+07      -     -    2s
     0     0 4.0812e+07    0  201          - 4.0812e+07      -     -    2s
     0     0 4.0812e+07    0  201          - 4.0812e+07      -     -    2s
     0     0 4.0812e+07    0  140          - 4.0812e+07      -     -    2s
     0     0 4.0812e+07    0  182          - 4.0812e+07      -     -    3s
     0     0 4.0812e+07    0  152          - 4.0812e+07      -     -    3s
     0     0 4.0812e+07    0  138          - 4.0812e+07      -     -    4s
     0     0 4.0812e+07    0  193          - 4.0812e+07      -     -    5s
     0     0 4.0812e+07    0  155          - 4.0812e+07      -     -    5s
     0     0 4.0812e+07    0  155          - 4.0812e+07      -     -    5s
----------------------------------------------------------------
     0     2 4.0812e+07    0  153          - 4.0812e+07      -     -   15s
   232   288 4.1278e+07   49  102  

     0     0 459282.000    0  226          - 459282.000      -     -    1s
     0     0 459304.000    0  225          - 459304.000      -     -    4s
     0     0 459304.000    0  224          - 459304.000      -     -    4s
     0     0 459304.000    0  256          - 459304.000      -     -    5s
     0     0 459304.000    0  256          - 459304.000      -     -    5s
     0     0 459304.000    0  249          - 459304.000      -     -    5s
     0     0 459304.000    0  227          - 459304.000      -     -    6s
     0     0 459304.000    0  260          - 459304.000      -     -    9s
     0     0 459304.000    0  159          - 459304.000      -     -   10s
     0     0 459304.000    0  159          - 459304.000      -     -   10s
     0     0 459304.000    0  162          - 459304.000      -     -   11s
     0     0 459304.000    0  161          - 459304.000      -     -   11s
     0     2 459304.000    0  133          - 459304.000      -     -   36s
   135   173 459639.081  

Variable types: 0 continuous, 8190 integer (8100 binary)

Root relaxation: objective 4.579655e+07, 411 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.5797e+07    0  122          - 4.5797e+07      -     -    0s
     0     0 4.5962e+07    0  224          - 4.5962e+07      -     -    0s
     0     0 4.5978e+07    0  228          - 4.5978e+07      -     -    0s
     0     0 4.5978e+07    0  221          - 4.5978e+07      -     -    0s
     0     0 4.5979e+07    0  239          - 4.5979e+07      -     -    1s
     0     0 4.5980e+07    0  228          - 4.5980e+07      -     -    1s
     0     0 4.5980e+07    0  220          - 4.5980e+07      -     -    1s
     0     0 4.5980e+07    0  215          - 4.5980e+07      -     -    1s
     0     0 4.5980e+07    0  215          - 4.5980e+07      -     -    1s
     0     0 4.5980e+07    

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10001 rows, 10100 columns and 49403 nonzeros
Model fingerprint: 0xf415801d
Variable types: 0 continuous, 10100 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.13s
Presolved: 10001 rows, 10100 columns, 49403 nonzeros
Variable types: 0 continuous, 10100 integer (10000 binary)

Root relaxation: objective 5.069919e+05, 510 iterations, 0.09 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 506991.890    0  137          - 506991.890      -     -   


Explored 502 nodes (28913 simplex iterations) in 80.41 seconds (39.90 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 5.15257e+06 5.15993e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.152573000000e+06, best bound 5.152573000000e+06, gap 0.0000%

User-callback calls 3911, time in user-callback 0.04 sec
Set parameter TimeLimit to value 600



---------------------------------------------------------------
Execução:  12
---------------------------------------------------------------



Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10001 rows, 10100 columns and 49403 nonzeros
Model fingerprint: 0x03c8acc7
Variable types: 0 continuous, 10100 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+05, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS 

         to avoid numerical issues.
Presolve time: 0.15s
Presolved: 12101 rows, 12210 columns, 59843 nonzeros
Variable types: 0 continuous, 12210 integer (12100 binary)

Root relaxation: objective 5.581685e+05, 596 iterations, 0.11 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 558168.491    0  170          - 558168.491      -     -    1s
     0     0 559935.809    0  300          - 559935.809      -     -    2s
     0     0 559962.855    0  277          - 559962.855      -     -    8s
     0     0 559962.891    0  279          - 559962.891      -     -    8s
     0     0 560017.000    0  321          - 560017.000      -     -    9s
     0     0 560017.000    0  321          - 560017.000      -     -   11s
     0     0 560017.000    0  323          - 560017.000      -     -   12s
     0     0 560017.000    0  323          - 560017.000      -   

     0     0 5.5949e+07    0  272          - 5.5949e+07      -     -    3s
     0     0 5.5956e+07    0  243          - 5.5956e+07      -     -    9s
     0     0 5.5956e+07    0  253          - 5.5956e+07      -     -    9s
     0     0 5.5956e+07    0  320          - 5.5956e+07      -     -    9s
     0     0 5.5956e+07    0  321          - 5.5956e+07      -     -    9s
     0     0 5.5956e+07    0  304          - 5.5956e+07      -     -   10s
     0     0 5.5956e+07    0  324          - 5.5956e+07      -     -   10s
     0     0 5.5956e+07    0  221          - 5.5956e+07      -     -   11s
     0     0 5.5956e+07    0  310          - 5.5956e+07      -     -   16s
     0     0 5.5956e+07    0  306          - 5.5956e+07      -     -   17s
     0     0 5.5956e+07    0  305          - 5.5956e+07      -     -   17s
     0     0 5.5956e+07    0  305          - 5.5956e+07      -     -   18s
     0     0 5.5956e+07    0  305          - 5.5956e+07      -     -   18s
     0     0 5.5956e+07  

## Branch direction

In [10]:
import gurobipy as gp

cont = 0

solucoes_DD = []
tempo_DD = []

for k in tamanhos:
  n = list(range(k))
  m = list(range(1, k))

  model = gp.Model()
  #Limitar a busca em 10 minutos
  model.setParam(gp.GRB.Param.TimeLimit, 600)

  # Insere as variáveis de decisão
  x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
  u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

  # Define a função objetivo
  model.setObjective(
      gp.quicksum(x[i, j] * costs_matrix[cont][i][j] for i in n for j in n),
      sense=gp.GRB.MINIMIZE)

  # Restrição 1
  c1 = model.addConstrs(
      gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

  # Restrição 2
  c2 = model.addConstrs(
      gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

  # Restrição 3
  c3 = model.addConstrs(u[i] - u[j] + k * x[i, j] <= k-1 
                            for i in m 
                            for j in n 
                            if (i != j))

    
  # Branch Direction Down
  model.setParam(gp.GRB.Param.BranchDir, -1)
  model.optimize()

  cont = cont+1

  solucoes_DD.append(model.objVal)
  tempo_DD.append(model.Runtime)
    
  print()
  print()
  print()
  print('---------------------------------------------------------------')
  print("Execução: ", cont)
  print('---------------------------------------------------------------')
  print()
  print()
  print()

Set parameter TimeLimit to value 600
Set parameter BranchDir to value -1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0xb3a2210d
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 79 rows and 1 columns
Presolve time: 0.05s
Presolved: 6322 rows, 6479 columns, 31286 nonzeros
Variable types: 0 continuous, 6479 integer (6400 binary)

Root relaxation: objective 4.081044e+05, 371 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

* 1901   789              17    4111507.0000 4110913.39  0.01%  52.4   45s

Cutting planes:
  Learned: 32
  Gomory: 36
  MIR: 31
  StrongCG: 1
  Flow cover: 172
  Inf proof: 1
  Zero half: 19
  RLT: 2

Explored 2260 nodes (118561 simplex iterations) in 48.76 seconds (28.13 work units)
Thread count was 8 (of 8 available processors)

Solution count 8: 4.11151e+06 4.11173e+06 4.11291e+06 ... 8e+18

Optimal solution found (tolerance 1.00e-04)
Best objective 4.111507000000e+06, best bound 4.111098920113e+06, gap 0.0099%



---------------------------------------------------------------
Execução:  2
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter BranchDir to value -1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0x6a6c2fa8
Variable types: 0 continuous, 648

Set parameter BranchDir to value -1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8101 rows, 8190 columns and 39963 nonzeros
Model fingerprint: 0xb3e8a0c2
Variable types: 0 continuous, 8190 integer (8100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 89 rows and 1 columns
Presolve time: 0.07s
Presolved: 8012 rows, 8189 columns, 39696 nonzeros
Variable types: 0 continuous, 8189 integer (8100 binary)

Root relaxation: objective 4.572328e+05, 432 iterations, 0.03 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

    

     0     0 4580439.00    0   84 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00    0   84 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00    0  100 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00    0  104 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00    0  104 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00    0  109 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00    0  109 4593004.00 4580439.00  0.27%     -    5s
     0     0 4580439.00    0  112 4593004.00 4580439.00  0.27%     -    5s
     0     0 4580439.00    0  113 4593004.00 4580439.00  0.27%     -    5s
     0     0 4580439.00    0  110 4593004.00 4580439.00  0.27%     -    5s
     0     2 4580439.00    0   84 4593004.00 4580439.00  0.27%     -    6s
*   21    22               6    4580691.0000 4580440.19  0.01%   107    7s

Cutting planes:
  Learned: 37
  Gomory: 3
  MIR: 101
  StrongCG: 14

Explored 23 nodes (7728 simple

  1161   818 4.5982e+07   17  209 4.6008e+07 4.5982e+07  0.06%  68.1   46s
  1329   956 4.5990e+07   39   46 4.6008e+07 4.5982e+07  0.06%  69.8   50s
  1587  1081 4.5999e+07   66  118 4.6008e+07 4.5982e+07  0.06%  69.4   55s
* 1668  1097              78    4.600690e+07 4.5982e+07  0.05%  69.0   56s
H 1820  1012                    4.600074e+07 4.5982e+07  0.04%  68.6   59s
  1829  1122 4.5989e+07   23  235 4.6001e+07 4.5982e+07  0.04%  68.6   60s
  2481  1509 4.5982e+07   49  200 4.6001e+07 4.5982e+07  0.04%  65.4   65s
* 2541  1450              39    4.600068e+07 4.5982e+07  0.04%  64.4   65s
* 2852  1395              99    4.599646e+07 4.5982e+07  0.03%  62.9   67s
* 2881   879              42    4.599012e+07 4.5982e+07  0.02%  62.9   67s
  3349   706     cutoff   39      4.5990e+07 4.5982e+07  0.02%  61.5   70s
* 3364   630              36    4.598938e+07 4.5982e+07  0.02%  61.3   70s
  3966   753     cutoff   30      4.5989e+07 4.5983e+07  0.01%  60.4   77s
  4343   833     cutoff  

     0     0 508283.460    0  254          - 508283.460      -     -    1s
H    0     0                    508355.00000 508283.460  0.01%     -    2s
     0     0     cutoff    0      508355.000 508355.000  0.00%     -    2s

Cutting planes:
  Learned: 38
  MIR: 62
  Zero half: 1

Explored 1 nodes (1140 simplex iterations) in 2.30 seconds (0.70 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 508355 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.083550000000e+05, best bound 5.083550000000e+05, gap 0.0000%



---------------------------------------------------------------
Execução:  9
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter BranchDir to value -1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10001 rows, 10100 columns and 49403 nonzeros
Model fingerprint: 0x20710

     0     0 5.1040e+08    0  249          - 5.1040e+08      -     -    2s
H    0     0                    5.135407e+08 5.1040e+08  0.61%     -    2s
     0     0 5.1040e+08    0  298 5.1354e+08 5.1040e+08  0.61%     -    2s
     0     0 5.1040e+08    0  141 5.1354e+08 5.1040e+08  0.61%     -    2s
     0     0 5.1040e+08    0  308 5.1354e+08 5.1040e+08  0.61%     -    3s
     0     0 5.1047e+08    0  205 5.1354e+08 5.1047e+08  0.60%     -    4s
     0     0 5.1047e+08    0  286 5.1354e+08 5.1047e+08  0.60%     -    4s
     0     0 5.1047e+08    0  292 5.1354e+08 5.1047e+08  0.60%     -    4s
     0     0 5.1047e+08    0  293 5.1354e+08 5.1047e+08  0.60%     -    8s
     0     0 5.1047e+08    0  284 5.1354e+08 5.1047e+08  0.60%     -    9s
     0     0 5.1047e+08    0  229 5.1354e+08 5.1047e+08  0.60%     -    9s
     0     0 5.1047e+08    0  280 5.1354e+08 5.1047e+08  0.60%     -    9s
     0     0 5.1047e+08    0   65 5.1354e+08 5.1047e+08  0.60%     -   10s
     0     0 5.1047e+08  


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5581193.15    0  180          - 5581193.15      -     -    0s
     0     0 5603200.78    0  283          - 5603200.78      -     -    2s
     0     0 5603973.98    0  294          - 5603973.98      -     -    2s
     0     0 5603973.98    0  295          - 5603973.98      -     -    2s
     0     0 5604477.65    0  331          - 5604477.65      -     -    3s
     0     0 5604479.30    0  325          - 5604479.30      -     -    4s
     0     0 5604621.00    0  307          - 5604621.00      -     -    5s
     0     0 5604621.00    0  318          - 5604621.00      -     -    9s
     0     0 5604621.00    0  311          - 5604621.00      -     -    9s
     0     0 5604621.00    0  307          - 5604621.00      -     -   12s
     0     0 5604621.00    0   51          - 5604621.00      -     -   12s
     0     0 5604621.00

  1180  1054 5.5960e+07   13  140 5.6073e+07 5.5957e+07  0.21%  82.1   78s
  1183  1056 5.5985e+07   49  268 5.6073e+07 5.5957e+07  0.21%  81.9   81s
  1188  1060 5.6045e+07  133  322 5.6073e+07 5.5957e+07  0.21%  81.5   85s
  1194  1064 5.5985e+07   52  258 5.6073e+07 5.5957e+07  0.21%  81.1   94s
  1195  1064 5.6023e+07  132  271 5.6073e+07 5.5957e+07  0.21%  81.0   95s
  1201  1068 5.5967e+07   45  272 5.6073e+07 5.5957e+07  0.21%  80.6  101s
  1202  1072 5.5957e+07   31  288 5.6073e+07 5.5957e+07  0.21%  85.4  105s
  1208  1078 5.5957e+07   33  280 5.6073e+07 5.5957e+07  0.21%  85.5  110s
  1228  1091 5.5963e+07   38   83 5.6073e+07 5.5957e+07  0.21%  86.5  141s
  1304  1168 5.5974e+07   57  260 5.6073e+07 5.5957e+07  0.21%  88.2  146s
  1448  1268 5.6005e+07   92  191 5.6073e+07 5.5957e+07  0.21%  88.0  150s
  1604  1343 5.6049e+07  130  137 5.6073e+07 5.5957e+07  0.21%  87.9  155s
  1889  1537 5.6015e+07   93  235 5.6073e+07 5.5957e+07  0.21%  86.4  160s
H 2004  1425             


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5.5711e+08    0  158          - 5.5711e+08      -     -    1s
     0     0 5.5861e+08    0  283          - 5.5861e+08      -     -    3s
     0     0 5.5868e+08    0  278          - 5.5868e+08      -     -    5s
     0     0 5.5868e+08    0  283          - 5.5868e+08      -     -    5s
     0     0 5.5876e+08    0  328          - 5.5876e+08      -     -    6s
     0     0 5.5877e+08    0  333          - 5.5877e+08      -     -    8s
     0     0 5.5877e+08    0  333          - 5.5877e+08      -     -    8s
     0     0 5.5886e+08    0  323          - 5.5886e+08      -     -   10s
     0     0 5.5886e+08    0  329          - 5.5886e+08      -     -   12s
     0     0 5.5886e+08    0  330          - 5.5886e+08      -     -   12s
     0     0 5.5886e+08    0  330          - 5.5886e+08      -     -   13s
     0     0 5.5886e+08

In [11]:
import gurobipy as gp

cont = 0

solucoes_DU = []
tempo_DU = []

for k in tamanhos:
  n = list(range(k))
  m = list(range(1, k))

  model = gp.Model()
  #Limitar a busca em 10 minutos
  model.setParam(gp.GRB.Param.TimeLimit, 600)

  # Insere as variáveis de decisão
  x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
  u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

  # Define a função objetivo
  model.setObjective(
      gp.quicksum(x[i, j] * costs_matrix[cont][i][j] for i in n for j in n),
      sense=gp.GRB.MINIMIZE)

  # Restrição 1
  c1 = model.addConstrs(
      gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

  # Restrição 2
  c2 = model.addConstrs(
      gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

  # Restrição 3
  c3 = model.addConstrs(u[i] - u[j] + k * x[i, j] <= k-1 
                            for i in m 
                            for j in n 
                            if (i != j))

    
  # Branch Direction Up
  model.setParam(gp.GRB.Param.BranchDir, 1)
  model.optimize()

  cont = cont+1

  solucoes_DU.append(model.objVal)
  tempo_DU.append(model.Runtime)
    
  print()
  print()
  print()
  print('---------------------------------------------------------------')
  print("Execução: ", cont)
  print('---------------------------------------------------------------')
  print()
  print()
  print()

Set parameter TimeLimit to value 600
Set parameter BranchDir to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0xb3a2210d
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 79 rows and 1 columns
Presolve time: 0.07s
Presolved: 6322 rows, 6479 columns, 31286 nonzeros
Variable types: 0 continuous, 6479 integer (6400 binary)

Root relaxation: objective 4.081044e+05, 371 iterations, 0.05 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 

   178   133 4112759.09   12  127 4122650.00 4110701.00  0.29%  63.4   15s
*  339   181              19    4121595.0000 4110701.00  0.26%  56.0   16s
*  440   220              14    4116873.0000 4110701.50  0.15%  54.4   17s
   675   291 4115688.18   14   90 4116873.00 4110787.41  0.15%  54.7   20s
*  766   238              11    4111507.0000 4110788.39  0.02%  54.1   20s

Cutting planes:
  Learned: 35
  Gomory: 57
  MIR: 248
  StrongCG: 6
  Mod-K: 2

Explored 1359 nodes (64501 simplex iterations) in 23.17 seconds (11.33 work units)
Thread count was 8 (of 8 available processors)

Solution count 9: 4.11151e+06 4.11687e+06 4.1216e+06 ... 4.1961e+06

Optimal solution found (tolerance 1.00e-04)
Best objective 4.111507000000e+06, best bound 4.111188438636e+06, gap 0.0077%



---------------------------------------------------------------
Execução:  2
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter BranchDir to value 1
Gur

Thread count was 8 (of 8 available processors)

Solution count 10: 4.10718e+08 4.1072e+08 4.10733e+08 ... 4.3087e+08

Optimal solution found (tolerance 1.00e-04)
Best objective 4.107178020000e+08, best bound 4.106913315558e+08, gap 0.0064%



---------------------------------------------------------------
Execução:  4
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter BranchDir to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8101 rows, 8190 columns and 39963 nonzeros
Model fingerprint: 0xb3e8a0c2
Variable types: 0 continuous, 8190 integer (8100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerica

     0     0 4580439.00    0  251          - 4580439.00      -     -    1s
H    0     0                    4593004.0000 4580439.00  0.27%     -    2s
     0     0 4580439.00    0  228 4593004.00 4580439.00  0.27%     -    2s
     0     0 4580439.00    0   84 4593004.00 4580439.00  0.27%     -    3s
     0     0 4580439.00    0  113 4593004.00 4580439.00  0.27%     -    3s
     0     0 4580439.00    0  249 4593004.00 4580439.00  0.27%     -    3s
     0     0 4580439.00    0  231 4593004.00 4580439.00  0.27%     -    3s
     0     0 4580439.00    0  234 4593004.00 4580439.00  0.27%     -    3s
     0     0 4580439.00    0  241 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00    0  240 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00    0  195 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00    0  191 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00    0   84 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00  

Set parameter BranchDir to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8101 rows, 8190 columns and 39963 nonzeros
Model fingerprint: 0xb639b0ef
Variable types: 0 continuous, 8190 integer (8100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+06, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 89 rows and 1 columns
Presolve time: 0.07s
Presolved: 8012 rows, 8189 columns, 39696 nonzeros
Variable types: 0 continuous, 8189 integer (8100 binary)

Root relaxation: objective 4.595493e+08, 465 iterations, 0.04 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     

     0     0 5117032.00    0  262          - 5117032.00      -     -    6s
     0     0 5117032.00    0  262          - 5117032.00      -     -    6s
     0     0 5117032.00    0  285          - 5117032.00      -     -    7s
     0     0 5117032.00    0  289          - 5117032.00      -     -    8s
     0     0 5117032.00    0  293          - 5117032.00      -     -    8s
     0     0 5117032.00    0  293          - 5117032.00      -     -    8s
H    0     0                    5295674.0000 5117032.00  3.37%     -    9s
H    0     0                    5280834.0000 5117032.00  3.10%     -    9s
H    0     0                    5272126.0000 5117032.00  2.94%     -    9s
H    0     0                    5243451.0000 5117032.00  2.41%     -    9s
     0     0 5117032.00    0  267 5243451.00 5117032.00  2.41%     -    9s
     0     0 5117032.00    0  294 5243451.00 5117032.00  2.41%     -   11s
     0     0 5117032.00    0  232 5243451.00 5117032.00  2.41%     -   12s
H    0     0             

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 99 rows and 1 columns
Presolve time: 0.12s
Presolved: 9902 rows, 10099 columns, 49106 nonzeros
Variable types: 0 continuous, 10099 integer (10000 binary)

Root relaxation: objective 5.084336e+08, 480 iterations, 0.07 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5.0843e+08    0  140          - 5.0843e+08      -     -    1s
     0     0 5.1033e+08    0  259          - 5.1033e+08      -     -    2s
     0     0 5.1040e+08    0  241          - 5.1040e+08      -     -    3s
     0     0 5.1040e+08    0  249          - 5.1040e+08      -     -    3s
H    0     0                    5.135407e+08 5.1040e+08  0.61%     -    4s
     0     0 5.1040e+08    0  298 5.1

     0     0 5603973.98    0  295          - 5603973.98      -     -    2s
     0     0 5604477.65    0  331          - 5604477.65      -     -    3s
     0     0 5604479.30    0  325          - 5604479.30      -     -    3s
     0     0 5604621.00    0  307          - 5604621.00      -     -    5s
     0     0 5604621.00    0  318          - 5604621.00      -     -    8s
     0     0 5604621.00    0  311          - 5604621.00      -     -    8s
     0     0 5604621.00    0  307          - 5604621.00      -     -   10s
     0     0 5604621.00    0   51          - 5604621.00      -     -   10s
     0     0 5604621.00    0   54          - 5604621.00      -     -   11s
     0     0 5604621.00    0   48          - 5604621.00      -     -   11s
     0     0 5604621.00    0   29          - 5604621.00      -     -   12s
H    0     0                    1.100000e+19 5604621.00   100%     -   13s
     0     0 5604621.00    0   29 1.1000e+19 5604621.00   100%     -   13s
H    0     0             

  5124   968 5.5977e+07   64  247 5.5982e+07 5.5957e+07  0.04%  88.1  171s
  5438  1011 5.5976e+07   68  191 5.5982e+07 5.5957e+07  0.04%  86.6  176s
  6224  1087 5.5974e+07   99  244 5.5982e+07 5.5957e+07  0.04%  83.8  184s
  6472  1124     cutoff   78      5.5982e+07 5.5957e+07  0.04%  82.7  188s
  7027  1185 5.5976e+07   82  271 5.5982e+07 5.5957e+07  0.04%  80.2  192s
* 7193   988              81    5.597708e+07 5.5957e+07  0.04%  79.3  192s
  7589   999     cutoff   90      5.5977e+07 5.5957e+07  0.04%  78.4  198s
  7910  1058 5.5968e+07  100  225 5.5977e+07 5.5957e+07  0.04%  77.1  203s
  8449  1092     cutoff  126      5.5977e+07 5.5957e+07  0.04%  76.7  208s
  8948  1103     cutoff  135      5.5977e+07 5.5958e+07  0.03%  74.8  213s
* 9130  1038              72    5.597602e+07 5.5958e+07  0.03%  73.9  214s
  9497  1079 5.5958e+07  156  258 5.5976e+07 5.5958e+07  0.03%  72.5  219s
 10108  1163     cutoff   90      5.5976e+07 5.5958e+07  0.03%  71.5  226s
*10720   985             

     0     0 5.5886e+08    0  323          - 5.5886e+08      -     -    4s
     0     0 5.5886e+08    0  329          - 5.5886e+08      -     -    5s
     0     0 5.5886e+08    0  330          - 5.5886e+08      -     -    5s
     0     0 5.5886e+08    0  330          - 5.5886e+08      -     -    6s
     0     0 5.5886e+08    0  218          - 5.5886e+08      -     -    6s
     0     0 5.5886e+08    0  263          - 5.5886e+08      -     -    9s
     0     0 5.5891e+08    0  227          - 5.5891e+08      -     -    9s
     0     0 5.5891e+08    0  260          - 5.5891e+08      -     -   13s
     0     0 5.5891e+08    0  270          - 5.5891e+08      -     -   13s
     0     0 5.5891e+08    0  270          - 5.5891e+08      -     -   13s
H    0     0                    5.748242e+08 5.5891e+08  2.77%     -   13s
H    0     0                    5.744203e+08 5.5891e+08  2.70%     -   13s
     0     0 5.5891e+08    0  187 5.7442e+08 5.5891e+08  2.70%     -   14s
     0     0 5.5891e+08  

## Cuts Control

In [12]:
import gurobipy as gp

cont = 0

solucoes_NC = []
tempo_NC = []

for k in tamanhos:
  n = list(range(k))
  m = list(range(1, k))

  model = gp.Model()
  #Limitar a busca em 10 minutos
  model.setParam(gp.GRB.Param.TimeLimit, 600)

  # Insere as variáveis de decisão
  x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
  u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

  # Define a função objetivo
  model.setObjective(
      gp.quicksum(x[i, j] * costs_matrix[cont][i][j] for i in n for j in n),
      sense=gp.GRB.MINIMIZE)

  # Restrição 1
  c1 = model.addConstrs(
      gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

  # Restrição 2
  c2 = model.addConstrs(
      gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

  # Restrição 3
  c3 = model.addConstrs(u[i] - u[j] + k * x[i, j] <= k-1 
                            for i in m 
                            for j in n 
                            if (i != j))

    
  # No Cuts
  model.setParam(gp.GRB.Param.Cuts, 0)
  model.optimize()

  cont = cont+1

  solucoes_NC.append(model.objVal)
  tempo_NC.append(model.Runtime)
    
  print()
  print()
  print()
  print('---------------------------------------------------------------')
  print("Execução: ", cont)
  print('---------------------------------------------------------------')
  print()
  print()
  print()

Set parameter TimeLimit to value 600
Set parameter Cuts to value 0
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0xb3a2210d
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 79 rows and 1 columns
Presolve time: 0.05s
Presolved: 6322 rows, 6479 columns, 31286 nonzeros
Variable types: 0 continuous, 6479 integer (6400 binary)

Root relaxation: objective 4.081044e+05, 371 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

 116414  5301     cutoff   35      409722.000 409592.975  0.03%  46.8  463s
 116482  4981     cutoff   36      409722.000 409593.100  0.03%  46.7  467s
 117682  4597 409598.388   21   92 409722.000 409596.692  0.03%  46.7  471s
 120487  3652 409637.780   33   78 409722.000 409605.350  0.03%  46.5  478s
 122004  3116     cutoff   27      409722.000 409611.177  0.03%  46.4  482s
 123467  2453     cutoff   26      409722.000 409618.175  0.03%  46.4  486s
 125086  1673 409629.038   26   18 409722.000 409627.600  0.02%  46.2  490s
 128101     0     cutoff   27      409722.000 409661.375  0.01%  46.0  495s

Explored 129042 nodes (5915738 simplex iterations) in 496.10 seconds (291.96 work units)
Thread count was 8 (of 8 available processors)

Solution count 4: 409722 409925 411796 421610 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.097220000000e+05, best bound 4.097220000000e+05, gap 0.0000%



---------------------------------------------------------------
Execução:  1
----

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.0669e+07    0   90          - 4.0669e+07      -     -    0s
     0     0 4.0669e+07    0   90          - 4.0669e+07      -     -    1s
H    0     0                    4.144444e+07 4.0669e+07  1.87%     -    3s
     0     2 4.0669e+07    0   90 4.1444e+07 4.0669e+07  1.87%     -    3s
H   29    34                    4.096454e+07 4.0670e+07  0.72%  41.4    8s
   276   292 4.0783e+07   50   65 4.0965e+07 4.0671e+07  0.72%  28.7   10s
  1071   968 4.0690e+07    4  112 4.0965e+07 4.0671e+07  0.72%  22.2   16s
H 1073   923                    4.085186e+07 4.0688e+07  0.40%  23.2   20s
  2100  1475 4.0803e+07   99   76 4.0852e+07 4.0688e+07  0.40%  32.3   25s
  3590  2172     cutoff   84      4.0852e+07 4.0688e+07  0.40%  31.4   30s
  5438  3380 4.0797e+07   48   86 4.0852e+07 4.0701e+07  0.37%  32.2   35s
  6363  3902 4.0764e+07   19   82 4.0852e+07 4.0708e+07  0.35%  33.3   41s
  7468  4498 4.0727e+0

 61145 16037 4.1070e+08   32   32 4.1072e+08 4.1013e+08  0.14%  51.1  220s
 62953 16275 4.1055e+08   27   96 4.1072e+08 4.1014e+08  0.14%  51.2  225s
 64896 16496 4.1061e+08   28   49 4.1072e+08 4.1015e+08  0.14%  51.2  231s
 66709 16705 infeasible   28      4.1072e+08 4.1015e+08  0.14%  51.2  236s
 68452 16904     cutoff   28      4.1072e+08 4.1016e+08  0.14%  51.3  241s
 70289 17134 infeasible   33      4.1072e+08 4.1017e+08  0.13%  51.4  246s
 71403 17265 4.1055e+08   28  119 4.1072e+08 4.1017e+08  0.13%  51.4  250s
 73236 17431 4.1066e+08   27   66 4.1072e+08 4.1018e+08  0.13%  51.4  255s
 75132 17632 4.1037e+08   25  103 4.1072e+08 4.1018e+08  0.13%  51.5  260s
 76905 17813 4.1060e+08   30   58 4.1072e+08 4.1019e+08  0.13%  51.4  266s
 78658 17981 infeasible   35      4.1072e+08 4.1019e+08  0.13%  51.5  271s
 80575 18128 4.1021e+08   26  109 4.1072e+08 4.1020e+08  0.13%  51.5  276s
 82286 18285     cutoff   28      4.1072e+08 4.1020e+08  0.13%  51.6  281s
 84059 18448 4.1060e+08  

  3366  2247 459855.436  260   73 461633.000 457513.278  0.89%  49.9   45s
  4286  2953 460384.467   66   54 461633.000 457559.700  0.88%  45.7   50s
H 5026  3393                    461543.00000 457563.389  0.86%  44.4   54s
  5053  3596     cutoff   59      461543.000 457563.600  0.86%  44.5   55s
H 5352  3562                    461361.00000 457672.222  0.80%  43.8   60s
H 5353  3105                    460862.00000 457672.222  0.69%  43.8   60s
  6255  4051 459423.667   33  121 460862.000 457794.200  0.67%  41.6   65s
  7372  5014 460210.883  191   93 460862.000 457863.959  0.65%  41.8   70s
  8286  5867 459081.939   28  121 460862.000 457914.261  0.64%  42.7   76s
  9443  7016 458675.800   19   84 460862.000 457944.388  0.63%  41.7   81s
 10327  7487 460308.433   93  119 460862.000 457952.511  0.63%  41.3   86s
 10329  7489 458963.533   28  119 460862.000 457952.778  0.63%  41.3   93s
 10330  7492 457952.778   22  122 460862.000 457952.778  0.63%  41.4  101s
 10389  7538 458304.844  

...

 59650 29087 5.1030e+08   28  121 5.1054e+08 5.0955e+08  0.19%  69.7  476s
 60251 29314 5.0991e+08   26   77 5.1054e+08 5.0956e+08  0.19%  69.8  480s
 61413 29811 5.1026e+08   33   70 5.1054e+08 5.0956e+08  0.19%  69.8  488s
 62056 30070 5.1051e+08   31   53 5.1054e+08 5.0956e+08  0.19%  69.8  491s
 62591 30300 5.1009e+08   28  158 5.1054e+08 5.0956e+08  0.19%  69.9  495s
 63135 30598 5.0994e+08   26   79 5.1054e+08 5.0956e+08  0.19%  70.0  500s
 64397 31150 5.1028e+08   33  193 5.1054e+08 5.0957e+08  0.19%  70.1  508s
 64983 31351 5.1038e+08   33  107 5.1054e+08 5.0957e+08  0.19%  70.1  511s
 65531 31577 5.1035e+08   28   88 5.1054e+08 5.0957e+08  0.19%  70.1  515s
 66830 32108 infeasible   30      5.1054e+08 5.0957e+08  0.19%  70.1  523s
 67487 32277 infeasible   35      5.1054e+08 5.0958e+08  0.19%  70.1  527s
 67997 32496 5.1026e+08   31   88 5.1054e+08 5.0958e+08  0.19%  70.2  531s
 68597 32767     cutoff   35      5.1054e+08 5.0958e+08  0.19%  70.2  535s
 69167 33010 5.1032e+08  

 26248 16062     cutoff   41      560021.000 558842.472  0.21%  73.8  331s
 26759 16400 559386.830   36  104 560021.000 558847.518  0.21%  73.8  337s
 27279 16729     cutoff   50      560021.000 558850.582  0.21%  74.0  343s
 27923 17053 559880.618   37   74 560021.000 558857.655  0.21%  74.0  349s
 28480 17338 559746.164   36   76 560021.000 558861.709  0.21%  74.1  355s
 29090 17663 559802.673   33  140 560021.000 558868.964  0.21%  74.2  361s
 29674 17985 559751.200   41  100 560021.000 558872.643  0.21%  74.3  367s
 30335 18336 559480.455   32  100 560021.000 558878.600  0.20%  74.4  373s
 30960 18695 559812.891   44   77 560021.000 558882.600  0.20%  74.6  380s
 31604 19007 559754.700   37  101 560021.000 558888.086  0.20%  74.6  386s
 32258 19348 559661.402   36  132 560021.000 558894.209  0.20%  74.6  392s
 32885 19609     cutoff   47      560021.000 558898.327  0.20%  74.9  397s
 33466 19904 559865.673   47   53 560021.000 558902.927  0.20%  74.9  402s
 34028 20165 559402.270  

 16930 10215 5604014.00   47   93 5604791.00 5591332.40  0.24%  72.1  209s
 17440 10518     cutoff   43      5604791.00 5591392.90  0.24%  72.4  215s
 18047 10738 infeasible   44      5604791.00 5591437.43  0.24%  72.7  220s
 18497 11031 5595271.24   26  110 5604791.00 5591489.47  0.24%  73.0  225s
 19014 11352     cutoff   48      5604791.00 5591523.41  0.24%  73.1  231s
 19564 11605 5603650.05   42  110 5604791.00 5591556.35  0.24%  73.3  237s
 20072 11830     cutoff   41      5604791.00 5591629.38  0.23%  73.6  245s
 20539 12110 5599387.21   37  117 5604791.00 5591671.53  0.23%  73.8  252s
 21112 12355 5601858.83   43  155 5604791.00 5591732.74  0.23%  73.8  259s
 21623 12686 5595509.42   28  127 5604791.00 5591819.60  0.23%  74.0  266s
 22201 13022 5603056.02   36  158 5604791.00 5591882.07  0.23%  74.0  273s
 22874 13313 5593623.12   24  142 5604791.00 5591953.45  0.23%  74.2  278s
 23507 13630 5595653.34   26  125 5604791.00 5592029.13  0.23%  74.5  284s
 24141 13924 5602805.97  

  4021  2795 5.5975e+07  459  129 5.6073e+07 5.5809e+07  0.47%  52.8  202s
  4125  2880 5.5979e+07  481  163 5.6073e+07 5.5809e+07  0.47%  52.9  205s
  4463  3128 5.5990e+07  547   54 5.6073e+07 5.5809e+07  0.47%  52.0  212s
  4649  3272 5.5997e+07  584   99 5.6073e+07 5.5809e+07  0.47%  51.3  216s
  4856  3362 5.6010e+07  628   95 5.6073e+07 5.5809e+07  0.47%  50.5  220s
  5249  3716 5.6048e+07   77  134 5.6073e+07 5.5812e+07  0.47%  50.6  227s
  5492  3912 5.5913e+07   30  147 5.6073e+07 5.5812e+07  0.47%  50.7  232s
  5708  4032 5.5959e+07   55  113 5.6073e+07 5.5812e+07  0.47%  51.4  237s
  5828  4187 5.5976e+07   68  185 5.6073e+07 5.5812e+07  0.47%  52.2  241s
  5983  4377 5.5978e+07   84  180 5.6073e+07 5.5812e+07  0.47%  52.9  246s
  6173  4514 5.5979e+07   99  147 5.6073e+07 5.5812e+07  0.47%  53.0  250s
  6493  4695 5.5982e+07  137  142 5.6073e+07 5.5812e+07  0.47%  54.0  280s
H 6495  4151                    5.602444e+07 5.5812e+07  0.38%  54.0  280s
  6497  4226     cutoff  

  4717  4404 5.5781e+08   28  106 5.6252e+08 5.5748e+08  0.90%  24.6  155s
  4914  4549 5.5895e+08   51  122 5.6252e+08 5.5748e+08  0.90%  26.1  160s
  5185  4743 5.5981e+08   82   67 5.6252e+08 5.5748e+08  0.90%  27.6  165s
  5447  4905 5.5994e+08  114   54 5.6252e+08 5.5748e+08  0.90%  28.3  170s
  5737  5041 5.5997e+08  144   49 5.6252e+08 5.5748e+08  0.90%  29.2  175s
  6055  5227 5.6048e+08  174   30 5.6252e+08 5.5748e+08  0.90%  30.4  180s
  6359  5447 5.6002e+08  199  121 5.6252e+08 5.5748e+08  0.90%  30.9  186s
  6618  5588 5.6055e+08  223   49 5.6252e+08 5.5748e+08  0.90%  31.2  191s
  6823  5771 5.6005e+08  243  102 5.6252e+08 5.5748e+08  0.90%  31.5  195s
  7223  6014 5.6032e+08  277   66 5.6252e+08 5.5748e+08  0.90%  31.4  202s
  7494  6219 5.6063e+08  299   47 5.6252e+08 5.5748e+08  0.90%  31.1  207s
  7782  6416 5.6166e+08  324   35 5.6252e+08 5.5748e+08  0.90%  31.2  211s
  8185  6633 5.6038e+08  355  122 5.6252e+08 5.5748e+08  0.90%  31.0  215s
  8673  7020 5.6042e+08  

In [13]:
import gurobipy as gp

cont = 0

solucoes_MC = []
tempo_MC = []

for k in tamanhos:
  n = list(range(k))
  m = list(range(1, k))

  model = gp.Model()
  #Limitar a busca em 10 minutos
  model.setParam(gp.GRB.Param.TimeLimit, 600)

  # Insere as variáveis de decisão
  x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
  u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

  # Define a função objetivo
  model.setObjective(
      gp.quicksum(x[i, j] * costs_matrix[cont][i][j] for i in n for j in n),
      sense=gp.GRB.MINIMIZE)

  # Restrição 1
  c1 = model.addConstrs(
      gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

  # Restrição 2
  c2 = model.addConstrs(
      gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

  # Restrição 3
  c3 = model.addConstrs(u[i] - u[j] + k * x[i, j] <= k-1 
                            for i in m 
                            for j in n 
                            if (i != j))

    
  # Moderate Cuts
  model.setParam(gp.GRB.Param.Cuts, 1)
  model.optimize()

  cont = cont+1

  solucoes_MC.append(model.objVal)
  tempo_MC.append(model.Runtime)
    
  print()
  print()
  print()
  print('---------------------------------------------------------------')
  print("Execução: ", cont)
  print('---------------------------------------------------------------')
  print()
  print()
  print()

Set parameter TimeLimit to value 600
Set parameter Cuts to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0xb3a2210d
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 79 rows and 1 columns
Presolve time: 0.06s
Presolved: 6322 rows, 6479 columns, 31286 nonzeros
Variable types: 0 continuous, 6479 integer (6400 binary)

Root relaxation: objective 4.081044e+05, 371 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

Set parameter Cuts to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0x6a6c2fa8
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+05, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 79 rows and 1 columns
Presolve time: 0.06s
Presolved: 6322 rows, 6479 columns, 31286 nonzeros
Variable types: 0 continuous, 6479 integer (6400 binary)

Root relaxation: objective 4.066881e+07, 418 iterations, 0.03 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    

*  774   428              20    4.108400e+08 4.1065e+08  0.05%  34.5   12s
  1031   507 4.1072e+08   60   93 4.1084e+08 4.1067e+08  0.04%  39.7   15s
  1040   513 4.1080e+08   59   17 4.1084e+08 4.1067e+08  0.04%  39.4   20s
H 1042   489                    4.107178e+08 4.1067e+08  0.01%  39.3   22s
  1048   494 4.1070e+08   19   75 4.1072e+08 4.1067e+08  0.01%  41.1   25s

Cutting planes:
  Learned: 30
  Gomory: 11
  MIR: 18
  StrongCG: 1
  Flow cover: 137
  Mod-K: 4
  RLT: 1

Explored 1057 nodes (49107 simplex iterations) in 26.13 seconds (17.02 work units)
Thread count was 8 (of 8 available processors)

Solution count 6: 4.10718e+08 4.1084e+08 4.10885e+08 ... 4.11195e+08

Optimal solution found (tolerance 1.00e-04)
Best objective 4.107178020000e+08, best bound 4.106775150633e+08, gap 0.0098%



---------------------------------------------------------------
Execução:  4
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set paramet

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4565357.71    0  103          - 4565357.71      -     -    0s
     0     0 4579604.18    0  218          - 4579604.18      -     -    1s
     0     0 4580422.16    0  222          - 4580422.16      -     -    1s
     0     0 4580429.22    0  234          - 4580429.22      -     -    1s
     0     0 4580429.22    0  225          - 4580429.22      -     -    1s
     0     0 4580439.00    0  251          - 4580439.00      -     -    1s
H    0     0                    4593004.0000 4580439.00  0.27%     -    2s
     0     0 4580439.00    0  228 4593004.00 4580439.00  0.27%     -    2s
     0     0 4580439.00    0   84 4593004.00 4580439.00  0.27%     -    3s
     0     0 4580439.00    0  113 4593004.00 4580439.00  0.27%     -    3s
     0     0 4580439.00    0  249 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.00    0  231 4593004.00 4580439.00  0.27%     -    4s
     0     0 4580439.0

     0     0 4.5980e+07    0  129 4.6006e+07 4.5980e+07  0.06%     -    8s
     0     0 4.5980e+07    0  226 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  246 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  253 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  227 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  204 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  236 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  233 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  128 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0  140 4.6006e+07 4.5980e+07  0.06%     -   10s
     0     0 4.5980e+07    0   36 4.6006e+07 4.5980e+07  0.06%     -   11s
     0     0 4.5980e+07    0  211 4.6006e+07 4.5980e+07  0.06%     -   11s
     0     0 4.5980e+07    0  119 4.6006e+07 4.5980e+07  0.06%     -   11s
     0     0 4.5980e+07  

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 506991.890    0  144          - 506991.890      -     -    0s
     0     0 508283.460    0  254          - 508283.460      -     -    1s
H    0     0                    508355.00000 508283.460  0.01%     -    1s
     0     0     cutoff    0      508355.000 508355.000  0.00%     -    1s

Cutting planes:
  Learned: 38
  MIR: 62
  Zero half: 1

Explored 1 nodes (1140 simplex iterations) in 1.51 seconds (0.70 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 508355 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.083550000000e+05, best bound 5.083550000000e+05, gap 0.0000%



---------------------------------------------------------------
Execução:  9
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter Cuts to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physic

     0     0 5.1136e+07    0  278 5.1296e+07 5.1136e+07  0.31%     -   19s
     0     0 5.1136e+07    0  246 5.1296e+07 5.1136e+07  0.31%     -   20s
     0     0 5.1136e+07    0  251 5.1296e+07 5.1136e+07  0.31%     -   20s
     0     0 5.1136e+07    0  232 5.1296e+07 5.1136e+07  0.31%     -   20s
     0     0 5.1136e+07    0  230 5.1296e+07 5.1136e+07  0.31%     -   20s
     0     2 5.1136e+07    0  103 5.1296e+07 5.1136e+07  0.31%     -   22s
    28    33 5.1136e+07    6  256 5.1296e+07 5.1136e+07  0.31%   144   26s
H   29    33                    5.117018e+07 5.1136e+07  0.07%   139   26s
*   62    62              14    5.113695e+07 5.1136e+07  0.00%   109   27s

Cutting planes:
  Learned: 41
  Gomory: 10
  MIR: 173

Explored 63 nodes (12635 simplex iterations) in 27.29 seconds (18.62 work units)
Thread count was 8 (of 8 available processors)

Solution count 5: 5.11369e+07 5.11702e+07 5.12958e+07 ... 5.21856e+07

Optimal solution found (tolerance 1.00e-04)
Best objective 5.11369450

     0     0 560017.000    0  192 563875.000 560017.000  0.68%     -   23s
     0     0 560017.000    0   59 563875.000 560017.000  0.68%     -   23s
     0     0 560017.000    0  168 563875.000 560017.000  0.68%     -   24s
     0     0 560017.000    0  296 563875.000 560017.000  0.68%     -   24s
     0     0 560017.000    0  280 563875.000 560017.000  0.68%     -   26s
     0     0 560017.000    0  283 563875.000 560017.000  0.68%     -   26s
     0     0 560017.000    0  325 563875.000 560017.000  0.68%     -   27s
     0     0 560017.000    0  326 563875.000 560017.000  0.68%     -   27s
     0     0 560017.000    0  328 563875.000 560017.000  0.68%     -   27s
     0     0 560017.000    0  326 563875.000 560017.000  0.68%     -   28s
     0     0 560017.000    0  171 563875.000 560017.000  0.68%     -   29s
     0     0 560017.000    0  240 563875.000 560017.000  0.68%     -   30s
     0     0 560017.000    0  116 563875.000 560017.000  0.68%     -   30s
     0     0 560017.000  

  1492  1363 5.5976e+07  163  230 5.5980e+07 5.5956e+07  0.04%  14.3   56s
  1496  1365 5.5970e+07   93  228 5.5980e+07 5.5957e+07  0.04%  14.2   61s
  1500  1371 5.5957e+07   23  331 5.5980e+07 5.5957e+07  0.04%  16.5   68s
  1518  1385 5.5957e+07   26  232 5.5980e+07 5.5957e+07  0.04%  18.0   70s
  1706  1557 5.5958e+07   45  136 5.5980e+07 5.5957e+07  0.04%  25.3   75s
  2078  1604 5.5960e+07   67  221 5.5980e+07 5.5957e+07  0.04%  30.8   80s
  2252  1647 5.5960e+07   79  204 5.5980e+07 5.5957e+07  0.04%  32.8   86s
* 2442  1495              71    5.597439e+07 5.5957e+07  0.03%  33.4   88s
* 2514  1417              62    5.597308e+07 5.5957e+07  0.03%  33.4   89s
  2596  1497     cutoff  105      5.5973e+07 5.5957e+07  0.03%  33.7   91s
  3340  1833 5.5958e+07   33  280 5.5973e+07 5.5957e+07  0.03%  35.1   97s
  3777  1750 5.5963e+07   34  185 5.5973e+07 5.5958e+07  0.03%  38.4  100s
  4250  1850 5.5972e+07   38  163 5.5973e+07 5.5958e+07  0.03%  40.3  106s
  4419  1844     cutoff  

In [14]:
import gurobipy as gp

cont = 0

solucoes_AC = []
tempo_AC = []

for k in tamanhos:
  n = list(range(k))
  m = list(range(1, k))

  model = gp.Model()
  #Limitar a busca em 10 minutos
  model.setParam(gp.GRB.Param.TimeLimit, 600)

  # Insere as variáveis de decisão
  x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
  u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

  # Define a função objetivo
  model.setObjective(
      gp.quicksum(x[i, j] * costs_matrix[cont][i][j] for i in n for j in n),
      sense=gp.GRB.MINIMIZE)

  # Restrição 1
  c1 = model.addConstrs(
      gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

  # Restrição 2
  c2 = model.addConstrs(
      gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

  # Restrição 3
  c3 = model.addConstrs(u[i] - u[j] + k * x[i, j] <= k-1 
                            for i in m 
                            for j in n 
                            if (i != j))

    
  # Aggressive Cuts
  model.setParam(gp.GRB.Param.Cuts, 2)
  model.optimize()

  cont = cont+1

  solucoes_AC.append(model.objVal)
  tempo_AC.append(model.Runtime)
    
  print()
  print()
  print()
  print('---------------------------------------------------------------')
  print("Execução: ", cont)
  print('---------------------------------------------------------------')
  print()
  print()
  print()

Set parameter TimeLimit to value 600
Set parameter Cuts to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0xb3a2210d
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 79 rows and 1 columns
Presolve time: 0.06s
Presolved: 6322 rows, 6479 columns, 31286 nonzeros
Variable types: 0 continuous, 6479 integer (6400 binary)

Root relaxation: objective 4.081044e+05, 371 iterations, 0.03 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be




---------------------------------------------------------------
Execução:  2
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter Cuts to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0x6a6c2fa8
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+05, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 79 rows and 1 columns
Presolve time: 0.04s
Presolved: 6322 rows, 6479 columns, 31286 nonzeros
Variable types: 0 continuous, 6479 integer (6400 binary)

Root relaxation: objective 4.066881e+07, 418 iterations, 0.02 seco

     0     0 4.1068e+08    0  236 4.1100e+08 4.1068e+08  0.08%     -    6s
     0     0 4.1068e+08    0  232 4.1100e+08 4.1068e+08  0.08%     -    7s
     0     0 4.1068e+08    0  233 4.1100e+08 4.1068e+08  0.08%     -    7s
     0     0 4.1068e+08    0  218 4.1100e+08 4.1068e+08  0.08%     -    7s
     0     0 4.1068e+08    0  224 4.1100e+08 4.1068e+08  0.08%     -    7s
     0     0 4.1068e+08    0  214 4.1100e+08 4.1068e+08  0.08%     -    7s
     0     0 4.1068e+08    0  214 4.1100e+08 4.1068e+08  0.08%     -    7s
     0     0 4.1069e+08    0  219 4.1100e+08 4.1069e+08  0.08%     -    8s
     0     0 4.1069e+08    0  218 4.1100e+08 4.1069e+08  0.08%     -    8s
     0     2 4.1069e+08    0  218 4.1100e+08 4.1069e+08  0.08%     -    8s
   141   211 4.1069e+08   39  175 4.1100e+08 4.1069e+08  0.08%  23.6   12s
*  533   367             111    4.107203e+08 4.1069e+08  0.01%  19.1   14s

Cutting planes:
  Learned: 34
  Gomory: 223
  Lift-and-project: 5
  MIR: 23
  StrongCG: 17
  Flow c




---------------------------------------------------------------
Execução:  5
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter Cuts to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8101 rows, 8190 columns and 39963 nonzeros
Model fingerprint: 0x15c9ca77
Variable types: 0 continuous, 8190 integer (8100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+04, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 89 rows and 1 columns
Presolve time: 0.07s
Presolved: 8012 rows, 8189 columns, 39696 nonzeros
Variable types: 0 continuous, 8189 integer (8100 binary)

Root relaxation: objective 4.565358e+06, 373 iterations, 0.03 seco

* 1117   479              13    4.598869e+07 4.5981e+07  0.02%  18.2   24s
  1159   257 4.5985e+07   43  109 4.5989e+07 4.5981e+07  0.02%  18.0   27s
  1166   262 4.5986e+07   71  186 4.5989e+07 4.5981e+07  0.02%  17.9   30s
  1171   265 4.5985e+07   77  213 4.5989e+07 4.5981e+07  0.02%  17.8   36s
  1174   267 4.5986e+07   76  228 4.5989e+07 4.5982e+07  0.01%  17.7   41s
  1178   270 4.5982e+07    4  229 4.5989e+07 4.5982e+07  0.01%  17.7   45s
  1212   302 4.5982e+07   20  258 4.5989e+07 4.5982e+07  0.01%  23.6   54s
  1220   305 4.5984e+07   20  198 4.5989e+07 4.5982e+07  0.01%  24.2   55s
  1505   352 4.5984e+07   32  145 4.5989e+07 4.5982e+07  0.01%  33.1   60s
* 1660   352              36    4.598593e+07 4.5982e+07  0.01%  34.8   61s

Cutting planes:
  Learned: 36
  Gomory: 90
  MIR: 21
  Flow cover: 98
  Zero half: 6

Explored 1792 nodes (68766 simplex iterations) in 61.63 seconds (42.83 work units)
Thread count was 8 (of 8 available processors)

Solution count 5: 4.59859e+07 4.


Root relaxation: objective 5.069919e+05, 443 iterations, 0.03 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 506991.890    0  144          - 506991.890      -     -    0s
H    0     0                    508355.00000 506991.890  0.27%     -    1s

Cutting planes:
  Learned: 34
  Gomory: 27
  Lift-and-project: 107
  Clique: 40
  Flow cover: 173
  Zero half: 2

Explored 1 nodes (766 simplex iterations) in 1.62 seconds (0.74 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 508355 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.083550000000e+05, best bound 5.083550000000e+05, gap 0.0000%



---------------------------------------------------------------
Execução:  9
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter Cuts to value 2
Gurobi 

  Mod-K: 3

Explored 965 nodes (52837 simplex iterations) in 30.50 seconds (25.25 work units)
Thread count was 8 (of 8 available processors)

Solution count 8: 5.11369e+07 5.11372e+07 5.11509e+07 ... 5.21856e+07

Optimal solution found (tolerance 1.00e-04)
Best objective 5.113694500000e+07, best bound 5.113612013586e+07, gap 0.0016%



---------------------------------------------------------------
Execução:  11
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter Cuts to value 2
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10001 rows, 10100 columns and 49403 nonzeros
Model fingerprint: 0x0440ce32
Variable types: 0 continuous, 10100 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+06, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
   

     0     0 5602860.09    0  283          - 5602860.09      -     -    2s
     0     0 5603857.15    0  292          - 5603857.15      -     -    2s
     0     0 5603857.15    0  290          - 5603857.15      -     -    2s
     0     0 5604621.00    0  243          - 5604621.00      -     -    3s
     0     0 5604621.00    0  285          - 5604621.00      -     -    5s
     0     0 5604621.00    0  264          - 5604621.00      -     -    7s
     0     0 5604621.00    0  220          - 5604621.00      -     -    8s
H    0     0                    5655597.0000 5604655.50  0.90%     -    9s
     0     0 5604655.50    0  130 5655597.00 5604655.50  0.90%     -    9s
H    0     0                    5654741.0000 5604655.50  0.89%     -   14s
     0     0 5604655.50    0  106 5654741.00 5604655.50  0.89%     -   14s
     0     0 5604655.50    0   85 5654741.00 5604655.50  0.89%     -   15s
H    0     0                    5621186.0000 5604655.50  0.29%     -   27s
     0     0 5604655.50  


Root relaxation: objective 5.571101e+08, 590 iterations, 0.07 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5.5711e+08    0  158          - 5.5711e+08      -     -    0s
     0     0 5.5868e+08    0  284          - 5.5868e+08      -     -    2s
     0     0 5.5868e+08    0  273          - 5.5868e+08      -     -    3s
     0     0 5.5868e+08    0  275          - 5.5868e+08      -     -    3s
     0     0 5.5878e+08    0  286          - 5.5878e+08      -     -    5s
     0     0 5.5886e+08    0  245          - 5.5886e+08      -     -    5s
     0     0 5.5886e+08    0  257          - 5.5886e+08      -     -    5s
     0     0 5.5895e+08    0  307          - 5.5895e+08      -     -    7s
     0     0 5.5895e+08    0  272          - 5.5895e+08      -     -    8s
H    0     0                    5.815487e+08 5.5895e+08  3.89%     -   12s
H    0   

In [15]:
import gurobipy as gp

cont = 0

solucoes_VAC = []
tempo_VAC = []

for k in tamanhos:
  n = list(range(k))
  m = list(range(1, k))

  model = gp.Model()
  #Limitar a busca em 10 minutos
  model.setParam(gp.GRB.Param.TimeLimit, 600)

  # Insere as variáveis de decisão
  x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
  u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

  # Define a função objetivo
  model.setObjective(
      gp.quicksum(x[i, j] * costs_matrix[cont][i][j] for i in n for j in n),
      sense=gp.GRB.MINIMIZE)

  # Restrição 1
  c1 = model.addConstrs(
      gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

  # Restrição 2
  c2 = model.addConstrs(
      gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

  # Restrição 3
  c3 = model.addConstrs(u[i] - u[j] + k * x[i, j] <= k-1 
                            for i in m 
                            for j in n 
                            if (i != j))

    
  # Very Aggressive Cuts
  model.setParam(gp.GRB.Param.Cuts, 3)
  model.optimize()

  cont = cont+1

  solucoes_VAC.append(model.objVal)
  tempo_VAC.append(model.Runtime)
    
  print()
  print()
  print()
  print('---------------------------------------------------------------')
  print("Execução: ", cont)
  print('---------------------------------------------------------------')
  print()
  print()
  print()

Set parameter TimeLimit to value 600
Set parameter Cuts to value 3
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0xb3a2210d
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 79 rows and 1 columns
Presolve time: 0.05s
Presolved: 6322 rows, 6479 columns, 31286 nonzeros
Variable types: 0 continuous, 6479 integer (6400 binary)

Root relaxation: objective 4.081044e+05, 371 iterations, 0.03 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

  Zero half: 7
  Mod-K: 4

Explored 1598 nodes (56213 simplex iterations) in 64.91 seconds (36.16 work units)
Thread count was 8 (of 8 available processors)

Solution count 10: 4.11151e+06 4.11394e+06 4.11714e+06 ... 4.21336e+06

Optimal solution found (tolerance 1.00e-04)
Best objective 4.111507000000e+06, best bound 4.111186675000e+06, gap 0.0078%



---------------------------------------------------------------
Execução:  2
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter Cuts to value 3
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0x6a6c2fa8
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+05, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00

     0     0 4.1066e+08    0  204 4.1098e+08 4.1066e+08  0.08%     -    7s
     0     0 4.1067e+08    0  196 4.1098e+08 4.1067e+08  0.08%     -    7s
     0     0 4.1067e+08    0  211 4.1098e+08 4.1067e+08  0.08%     -    7s
     0     0 4.1069e+08    0  125 4.1098e+08 4.1069e+08  0.07%     -    8s
     0     0 4.1069e+08    0  123 4.1098e+08 4.1069e+08  0.07%     -    8s
     0     0 4.1069e+08    0  132 4.1098e+08 4.1069e+08  0.07%     -    9s
     0     0 4.1069e+08    0  132 4.1098e+08 4.1069e+08  0.07%     -    9s
     0     2 4.1069e+08    0  132 4.1098e+08 4.1069e+08  0.07%     -   10s
H  279   305                    4.108352e+08 4.1069e+08  0.03%  26.3   12s
*  413   299              11    4.108074e+08 4.1069e+08  0.03%  20.5   12s
*  463   311              18    4.108058e+08 4.1069e+08  0.03%  25.2   13s
*  464   239               9    4.107178e+08 4.1069e+08  0.01%  25.4   13s

Cutting planes:
  Learned: 34
  Gomory: 127
  Clique: 2
  MIR: 21
  StrongCG: 5
  Flow cover: 161
 

Optimize a model with 8101 rows, 8190 columns and 39963 nonzeros
Model fingerprint: 0x15c9ca77
Variable types: 0 continuous, 8190 integer (8100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+04, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 89 rows and 1 columns
Presolve time: 0.06s
Presolved: 8012 rows, 8189 columns, 39696 nonzeros
Variable types: 0 continuous, 8189 integer (8100 binary)

Root relaxation: objective 4.565358e+06, 373 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4565357.71    0  103          - 4565357.71      -     -    0s
H    0     0                    4593004.0000 4580429.22  0.27%     -    2s
     0     0 4580439.

    32    54 4.5981e+07   10  284 4.6034e+07 4.5981e+07  0.11%  83.0   25s
   190   413 4.5982e+07   51  240 4.6034e+07 4.5981e+07  0.11%  22.2   30s
H  307   413                    4.598669e+07 4.5981e+07  0.01%  15.7   30s
*  761   267              22    4.598556e+07 4.5982e+07  0.01%  33.8   33s

Cutting planes:
  Learned: 39
  Gomory: 164
  Lift-and-project: 11
  MIR: 21
  StrongCG: 8
  Flow cover: 375
  Inf proof: 1
  Zero half: 65

Explored 880 nodes (39878 simplex iterations) in 33.28 seconds (21.82 work units)
Thread count was 8 (of 8 available processors)

Solution count 5: 4.59856e+07 4.59867e+07 4.60337e+07 ... 4.62836e+07

Optimal solution found (tolerance 1.00e-04)
Best objective 4.598555800000e+07, best bound 4.598202000000e+07, gap 0.0077%



---------------------------------------------------------------
Execução:  7
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter Cuts to value 3
Gurobi Optimizer vers

  MIR: 15
  StrongCG: 37
  Flow cover: 193
  Zero half: 3

Explored 1 nodes (2969 simplex iterations) in 3.60 seconds (1.77 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 508355 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.083550000000e+05, best bound 5.083550000000e+05, gap 0.0000%



---------------------------------------------------------------
Execução:  9
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter Cuts to value 3
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10001 rows, 10100 columns and 49403 nonzeros
Model fingerprint: 0x207100ee
Variable types: 0 continuous, 10100 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+04, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
      

Set parameter Cuts to value 3
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10001 rows, 10100 columns and 49403 nonzeros
Model fingerprint: 0x0440ce32
Variable types: 0 continuous, 10100 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+06, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 99 rows and 1 columns
Presolve time: 0.07s
Presolved: 9902 rows, 10099 columns, 49106 nonzeros
Variable types: 0 continuous, 10099 integer (10000 binary)

Root relaxation: objective 5.084336e+08, 480 iterations, 0.03 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   


Cutting planes:
  Learned: 34
  Gomory: 35
  Lift-and-project: 26
  MIR: 22
  Flow cover: 252
  Zero half: 97

Explored 1 nodes (3808 simplex iterations) in 25.81 seconds (12.94 work units)
Thread count was 8 (of 8 available processors)

Solution count 5: 560029 560115 560488 ... 564614

Optimal solution found (tolerance 1.00e-04)
Best objective 5.600290000000e+05, best bound 5.600178000000e+05, gap 0.0020%



---------------------------------------------------------------
Execução:  13
---------------------------------------------------------------



Set parameter TimeLimit to value 600
Set parameter Cuts to value 3
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12101 rows, 12210 columns and 59843 nonzeros
Model fingerprint: 0x3988ee64
Variable types: 0 continuous, 12210 integer (12100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+04, 1

  4163  1375 5.5963e+07   40  107 5.5966e+07 5.5959e+07  0.01%  50.6  204s
  4540  1429 5.5961e+07   28  188 5.5966e+07 5.5959e+07  0.01%  50.3  208s
  5076  1357 5.5961e+07   29   76 5.5966e+07 5.5959e+07  0.01%  49.9  212s
  5525  1245     cutoff   31      5.5966e+07 5.5960e+07  0.01%  51.3  216s
  5713  1278 5.5962e+07   39  139 5.5966e+07 5.5960e+07  0.01%  51.6  221s
  5917  1370 5.5962e+07   35  154 5.5966e+07 5.5960e+07  0.01%  52.2  225s
* 6369  1376              41    5.596637e+07 5.5960e+07  0.01%  53.4  229s
  6632  1391 5.5962e+07   38  224 5.5966e+07 5.5960e+07  0.01%  54.4  234s
  6770  1422     cutoff   41      5.5966e+07 5.5960e+07  0.01%  54.6  239s
  7054  1473 5.5961e+07   31  195 5.5966e+07 5.5961e+07  0.01%  54.6  244s

Cutting planes:
  Learned: 49
  Gomory: 145
  Lift-and-project: 3
  Cover: 1
  MIR: 45
  Flow cover: 240
  Inf proof: 2
  Zero half: 51
  Mod-K: 13

Explored 7484 nodes (418035 simplex iterations) in 244.65 seconds (240.23 work units)
Thread count w

# Guarda Resultados

In [18]:
Resultados = pd.DataFrame({'Instancias': urls,
                           'FO Pre-Solver': solucoes_pre, 
                           'FO Sem Pre-Solver': solucoes_no_pre, 
                           'Tempo COM': tempo_sol_pre, 
                           'Tempo SEM': tempo_sol_no_pre, 
                           'FO sem CallBacks': solucoes_NCB, 
                           'Tempo sem CallBack': tempo_NCB,
                           'FO CallBacks': solucoes_CB, 
                           'Tempo CallBack': tempo_CB, 
                           'FO Branch Dir (-1)': solucoes_DD, 
                           'Tempo Branch Dir (-1)': tempo_DD,
                           'FO Branch Dir (1)': solucoes_DU, 
                           'Tempo Branch Dir (1)': tempo_DU,
                           'FO No Cuts': solucoes_NC, 
                           'Tempo No Cuts': tempo_NC,
                           'FO Moderate Cuts': solucoes_MC, 
                           'Tempo Moderate Cuts': tempo_MC,
                           'FO Aggressive Cuts': solucoes_AC, 
                           'Tempo Aggressive Cuts': tempo_AC,
                           'FO Very Aggressive Cuts': solucoes_VAC, 
                           'Tempo Very Aggressive Cuts': tempo_VAC,
                          
                        })
                        

name = 'Resultados.xlsx'
print(name)
Resultados.to_excel(name)
print("Salvo ")

Resultados.xlsx
Salvo 


In [5]:
def NC_Callback(M,where):

   if where == gp.GRB.Callback.MIP:

      tempo = M.cbGet(gp.GRB.Callback.RUNTIME)

      melhor_solucao = M.cbGet(gp.GRB.Callback.MIP_OBJBST)

      melhor_bound = M.cbGet(gp.GRB.Callback.MIP_OBJBND)

      GAP = abs((melhor_solucao - melhor_bound)/melhor_solucao)

      if tempo > 20 and GAP < 0.1:

         M.terminate( )

In [6]:
import gurobipy as gp

cont = 0

solucoes_CBNC = []
tempo_CBNC = []

for k in tamanhos:
  n = list(range(k))
  m = list(range(1, k))

  model = gp.Model()
  #Limitar a busca em 10 minutos
  model.setParam(gp.GRB.Param.TimeLimit, 600)

  # Insere as variáveis de decisão
  x = model.addVars(n, n, vtype=gp.GRB.BINARY, name="x")
  u = model.addVars(n, vtype=gp.GRB.INTEGER, name="u")

  # Define a função objetivo
  model.setObjective(
      gp.quicksum(x[i, j] * costs_matrix[cont][i][j] for i in n for j in n),
      sense=gp.GRB.MINIMIZE)

  # Restrição 1
  c1 = model.addConstrs(
      gp.quicksum(x[i, j] for i in n) == 1 for j in n if (i != j))

  # Restrição 2
  c2 = model.addConstrs(
      gp.quicksum(x[j, i] for i in n) == 1 for j in n if (i != j))

  # Restrição 3
  c3 = model.addConstrs(u[i] - u[j] + k * x[i, j] <= k-1 
                            for i in m 
                            for j in n 
                            if (i != j))

    
  cont = cont+1

  print()
  print()
  print()
  print('---------------------------------------------------------------')
  print("Execução: ", cont)
  print('---------------------------------------------------------------')
  print()
  print()
  print()
  
  # Executa o modelo
  model.Params.lazyConstraints = 1
  model.setParam(gp.GRB.Param.Cuts, 0)
  model.optimize(NC_Callback)
    
    
  solucoes_CBNC.append(model.objVal)
  tempo_CBNC.append(model.Runtime)

Set parameter TimeLimit to value 600



---------------------------------------------------------------
Execução:  2
---------------------------------------------------------------



Set parameter LazyConstraints to value 1
Set parameter Cuts to value 0
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6401 rows, 6480 columns and 31523 nonzeros
Model fingerprint: 0xb3a2210d
Variable types: 0 continuous, 6480 integer (6400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [5e+03, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.03s
Presolved: 6401 rows, 6480 columns, 31523 nonzeros
Variable types: 0 continuous, 6480 integer (6400 binary)

Root relaxation: objective 4.081044e+05, 373 iterations, 0.02 se


Root relaxation: objective 4.081677e+08, 417 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.0817e+08    0  101          - 4.0817e+08      -     -    0s
     0     0 4.0817e+08    0   95          - 4.0817e+08      -     -    1s
H    0     0                    4.139558e+08 4.0817e+08  1.40%     -    1s
     0     2 4.0817e+08    0   95 4.1396e+08 4.0817e+08  1.40%     -    2s
H   30    35                    4.107951e+08 4.0820e+08  0.63%  44.8    5s
  1024   878 4.1007e+08   32   93 4.1080e+08 4.0847e+08  0.57%  37.7   11s
  1212  1020 4.0972e+08   38   79 4.1080e+08 4.0868e+08  0.51%  43.1   15s
  2932  1712 4.1036e+08   62  119 4.1080e+08 4.0909e+08  0.41%  41.7   20s
H 2974  1640                    4.107203e+08 4.0909e+08  0.40%  41.5   20s

Explored 3018 nodes (126847 simplex iterations) in 20.61 seconds (23.97 work units)

Set parameter TimeLimit to value 600



---------------------------------------------------------------
Execução:  9
---------------------------------------------------------------



Set parameter LazyConstraints to value 1
Set parameter Cuts to value 0
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8101 rows, 8190 columns and 39963 nonzeros
Model fingerprint: 0xb639b0ef
Variable types: 0 continuous, 8190 integer (8100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [5e+06, 1e+17]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve time: 0.05s
Presolved: 8101 rows, 8190 columns, 39963 nonzeros
Variable types: 0 continuous, 8190 integer (8100 binary)

Root relaxation: objective 4.595493e+08, 455 iterations, 0.02 se

*  715   713             117    5.206719e+07 5.0903e+07  2.24%  71.6   17s
   752   701     cutoff  128      5.2067e+07 5.0907e+07  2.23%  69.4   30s
H  754   427                    5.117252e+07 5.0907e+07  0.52%  69.2   30s
H  757   403                    5.115274e+07 5.0909e+07  0.48%  69.0   30s

Explored 760 nodes (53515 simplex iterations) in 30.91 seconds (30.12 work units)
Thread count was 8 (of 8 available processors)

Solution count 4: 5.11527e+07 5.11725e+07 5.20672e+07 5.2132e+07 

Solve interrupted
Best objective 5.115273600000e+07, best bound 5.090924260000e+07, gap 0.4760%

User-callback calls 4209, time in user-callback 0.03 sec
Set parameter TimeLimit to value 600



---------------------------------------------------------------
Execução:  13
---------------------------------------------------------------



Set parameter LazyConstraints to value 1
Set parameter Cuts to value 0
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 log


Root relaxation: objective 5.579386e+07, 577 iterations, 0.05 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5.5794e+07    0  131          - 5.5794e+07      -     -    0s
     0     0 5.5794e+07    0  131          - 5.5794e+07      -     -    3s
H    0     0                    5.631641e+07 5.5794e+07  0.93%     -    4s
     0     2 5.5794e+07    0  131 5.6316e+07 5.5794e+07  0.93%     -    6s
    27    32 5.5815e+07    7  165 5.6316e+07 5.5797e+07  0.92%  74.2   15s
   336   354 5.5843e+07   65  125 5.6316e+07 5.5797e+07  0.92%  47.6   20s

Explored 353 nodes (18182 simplex iterations) in 20.45 seconds (20.36 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 5.63164e+07 

Solve interrupted
Best objective 5.631641000000e+07, best bound 5.579714750909e+07, gap 0.9220%

User-callback calls 2487, time in user-callback 0

In [8]:
Resultados = pd.DataFrame({'Instancias': urls,
                           'FO CBNC': solucoes_CBNC, 
                           'Tempos CBNC': tempo_CBNC,
                          
                        })                      

name = 'CBNC.xlsx'
print(name)
Resultados.to_excel(name)
print("Salvo ")

CBNC.xlsx
Salvo 
